In [1]:
import numpy as np
from scipy.stats import norm
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

$\int f(y|x) \cdot \text{Normal}(x|\mu, \Sigma) dx$ \
where $y = Ax$ \
$f(y|x) = y.y^T$
\
We can write y|x because y is dependent on x(clearly)

In [2]:
mu = np.array([[1., 0., 1.5]])
scale = np.eye(3)
L = np.linalg.cholesky(scale)

x_dist = tfd.MultivariateNormalTriL(loc = mu, scale_tril = L)


In [3]:
from numpy.polynomial.hermite_e import hermegauss
from itertools import product

In [65]:
A = np.diag([1., 4, 2])
A

array([[1., 0., 0.],
       [0., 4., 0.],
       [0., 0., 2.]])

In [76]:
#p is pth Hermite Poly. n is order of Multivariate Normal Dist
p = 2
n = 3

unit_sigma_points, hermite_weights = hermegauss(p)
hermite_weights /= np.sum(hermite_weights)

unit_sigma_points = np.array(list(product(*(unit_sigma_points for _ in range(n))))).T  # Cartesian product
hermite_weights = np.array(list(product(*(hermite_weights for _ in range(n))))).prod(axis=1)

In [77]:
hermite_weights.shape #shape p^n

(8,)

In [78]:
scaled_sigma = L @ unit_sigma_points + mu.T

if f(.) = x. should get mean

In [79]:
np.dot(scaled_sigma, hermite_weights) #and we do.

array([1. , 0. , 1.5])

$y = Ax_i$

In [80]:
y = A @ scaled_sigma


In [81]:
integ = hermite_weights *y @ y.T

In [82]:
integ

array([[ 2.,  0.,  3.],
       [ 0., 16.,  0.],
       [ 3.,  0., 13.]])

$  \mathbb{E}[A x x^T A^T]= A\Sigma A^T + A\mu \mu^T A^T$

In [83]:
closed_form = A @ scale @ A.T + A @ mu.T @ mu @ A.T
closed_form

array([[ 2.,  0.,  3.],
       [ 0., 16.,  0.],
       [ 3.,  0., 13.]])

Just trying to do $ xx^T $. \
$\int xx^T \cdot \text{Normal}(x|\mu, \Sigma) dx \approx W_i (x_i x_i^T)$ 


In [84]:
hermite_weights * scaled_sigma @ scaled_sigma.T

array([[2.  , 0.  , 1.5 ],
       [0.  , 1.  , 0.  ],
       [1.5 , 0.  , 3.25]])

$  \mathbb{E}[x.x^T]= \Sigma + \mu \mu^T $

In [85]:
L + mu.T @ mu

array([[2.  , 0.  , 1.5 ],
       [0.  , 1.  , 0.  ],
       [1.5 , 0.  , 3.25]])

Using non-diagonal A

In [86]:
A = np.array([[1,2.,0],[0,1.,3], [1.2,3,7]])

In [87]:
y = A @ scaled_sigma

integ = hermite_weights *y @ y.T
integ

array([[  6.  ,   6.5 ,  18.9 ],
       [  6.5 ,  30.25,  76.65],
       [ 18.9 ,  76.65, 196.33]])

In [88]:
closed_form = A @ scale @ A.T + A @ mu.T @ mu @ A.T
closed_form

array([[  6.  ,   6.5 ,  18.9 ],
       [  6.5 ,  30.25,  76.65],
       [ 18.9 ,  76.65, 196.33]])

Using non-square A matrix (2,3) size

In [89]:
A = np.array([[1., 2, 2], [1.,0., 4]])

In [90]:
y = A @ scaled_sigma

integ = hermite_weights *y @ y.T
integ

array([[25., 37.],
       [37., 66.]])

In [91]:
closed_form = A @ scale @ A.T + A @ mu.T @ mu @ A.T
closed_form

array([[25., 37.],
       [37., 66.]])